In [35]:
cp *Grades-Summer*.csv CanvasGrades.csv

In [37]:
# Merge Beyond Labz Grades into Canvas Gradebook (Keep Higher Grades)
# Don't forget to publish to GitHub
# Purpose: To create a script that merges grades from a Beyond Labz csv into a Canvas csv for upload
# Problem outline: 
# Read the files, ignoring the second row in the canvas file
# Identify corresponding columns
# Map columns from beyond labz into canvas
# Merge the data, keeping the canvas grade, if higher
# Add the ignored row back in
# Create a new csv with only the columns that were merged to avoid introducing errors
# 
# Caution: Make sure you don't have extra students in either file. You should delete 
# Students who have dropped from Beyond Labz
# files are in /home/dperygin/Documents/mergegrades

import pandas as pd


# === Load Files ===
beyond_labz = pd.read_csv("gradebook-834.csv", skiprows=0)  # Skip student header row
canvas = pd.read_csv("CanvasGrades.csv", skiprows=[1])



In [39]:
beyond_labz.head(1)

,Name N=31,Username,Counting Molecules: Autograded Version 10 pts,Density of Solids and Liquids: Autograded Version 80 pts,Measuring Mass Pre-Lab 70 pts,Measuring Volume Pre-Lab 120 pts,Concepts in Acid-Base Titrations: Autograded Version 40 pts,Converting Concentrations to Different Units: Autograded Version 70 pts,Creating Chemical Compounds: Autograded Version 50 pts,Creating a Solution of Known Molarity: Autograded Version 40 pts,...,"Gas Laws: Pressure, Volume, Temperature and Moles: Autograded Version 70 pts",Heat of Fusion: Autograded Version 60 pts,Heat of Reaction: NaOH + HCl: Autograded Version 90 pts,Names and Formulas of Ionic Compounds: Autograded Version 10 pts,Phase Changes of Water: Autograded Version 70 pts,Rutherford's Experiment: Autograded Version 70 pts,Total 1060 pts,%,Comment,Instructor Note
0,"Aaron, Dakota",1687ce21-12e8-4fb2-973b-fe85aa4c7f1e,10,80,60.0,120.0,-,-,-,-,...,-,-,-,-,20,-,350/1060,33%,NaN,NaN


In [41]:
canvas.head(1)

,Student,ID,SIS User ID,SIS Login ID,Section,Module 0: Discussion -- Introduce Yourself (903005),Lab 0 Set up Beyond Labz (948405),Lab 1 Measuring Mass (948406),Lab 2 Measuring Volume (948411),Lab 3 Density of Solids and Liquids (948412),...,Current Points,Final Points,Current Score,Unposted Current Score,Final Score,Unposted Final Score,Current Grade,Unposted Current Grade,Final Grade,Unposted Final Grade
0,,62357,,@stu.jsu.edu,Summer 2025-SF,NaN,10.0,60.0,120.0,80.0,...,500.0,500.0,88.18,88.18,48.22,48.22,A,A,F,F


In [43]:
# === Define Column Mappings ===
# Beyond Labz column index -> Canvas column name
# Open the csv of the canvas file in Office, then copy the column header from the canvas column and paste it into the right column below
# Open the csv of the BL file and match the columns for the left one
merge_map = {
    'Measuring Mass Pre-Lab 70 pts': 'Lab 1 Measuring Mass (948406)',
    'Measuring Volume Pre-Lab 120 pts': 'Lab 2 Measuring Volume (948411)',
    'Density of Solids and Liquids: Autograded Version 80 pts': 'Lab 3 Density of Solids and Liquids (948412)',
    'Counting Molecules: Autograded Version 10 pts': 'Lab 4 Counting Molecules (948413)',
    'Phase Changes of Water: Autograded Version 70 pts': 'Lab 5 Phase Changes of Water (948414)',
    'Elements and the Periodic Table: Autograded Version 60 pts': 'Lab 6 Elements and the Periodic Table (948415)',
    'Names and Formulas of Ionic Compounds: Autograded Version 10 pts': 'Lab 7 Names and Formulas of Ionic Compounds (948416)',
    'Gas Laws: Pressure, Volume, Temperature and Moles: Autograded Version 70 pts': 'Lab 8 Gas Laws (948417)',
    'Converting Concentrations to Different Units: Autograded Version 70 pts': 'Lab 9 Converting Concentration to Different Units (948418)',
    'Creating a Solution of Known Molarity: Autograded Version 40 pts': 'Lab 10 Creating a Solution of Known Molarity (948407)',
    'Freezing Point Depression: Autograded Version 60 pts': 'Lab 11 Freezing Point Depression (948408)',
    'Endothermic vs. Exothermic: Autograded Version 20 pts': 'Lab 12 Endothermic vs Exothermic (948409)',
    'Concepts in Acid-Base Titrations: Autograded Version 40 pts': 'Lab 13 Concepts in Acid Base Titrations (948410)',
}



In [45]:
# === Merge Logic ===
# b, beyond labz
# c, canvas
# .items returns key:value pairs from dictionary
# min limits the search to the file with the smaller number of rows for safety 
# to avoid an index error. This should work fine as long as the number of students
# is the same in both files. If not, you should correct that before running this script anyway
# 'coerce' means replace non-numeric values with NaN if value is not numeric

updated_rows = []  # List to keep track of updated rows (optional)

for b_col, c_col in merge_map.items():
    if c_col not in canvas.columns:
        canvas[c_col] = pd.NA  # Create column if it doesn't exist to keep from throwing errors later
    for i in range(min(len(beyond_labz), len(canvas))):
        try:
            b_val = pd.to_numeric(beyond_labz.at[i, b_col], errors='coerce')
            c_val = pd.to_numeric(canvas.at[i, c_col], errors='coerce')
            if pd.notna(b_val):  # Only do the next part if b_val is not NaN
                if (pd.isna(c_val) or b_val > c_val) and b_val != 0:  # If Canvas has no grade (NaN) or if Beyond Labz's grade is better
                    canvas.at[i, c_col] = b_val
                    updated_rows.append((i, c_col))  # Log which row and column was updated
        except Exception:
            continue  # Skip errors

# Optional: print updated rows
print(f"✅ Updated {len(updated_rows)} grade entries.")
#print(updated_rows)

✅ Updated 42 grade entries.
[(6, 'Lab 4 Counting Molecules (948413)'), (6, 'Lab 5 Phase Changes of Water (948414)'), (9, 'Lab 5 Phase Changes of Water (948414)'), (11, 'Lab 5 Phase Changes of Water (948414)'), (12, 'Lab 5 Phase Changes of Water (948414)'), (15, 'Lab 5 Phase Changes of Water (948414)'), (20, 'Lab 5 Phase Changes of Water (948414)'), (22, 'Lab 5 Phase Changes of Water (948414)'), (23, 'Lab 5 Phase Changes of Water (948414)'), (24, 'Lab 5 Phase Changes of Water (948414)'), (26, 'Lab 5 Phase Changes of Water (948414)'), (27, 'Lab 5 Phase Changes of Water (948414)'), (29, 'Lab 5 Phase Changes of Water (948414)'), (30, 'Lab 5 Phase Changes of Water (948414)'), (3, 'Lab 6 Elements and the Periodic Table (948415)'), (6, 'Lab 6 Elements and the Periodic Table (948415)'), (9, 'Lab 6 Elements and the Periodic Table (948415)'), (11, 'Lab 6 Elements and the Periodic Table (948415)'), (12, 'Lab 6 Elements and the Periodic Table (948415)'), (15, 'Lab 6 Elements and the Periodic Table

In [47]:
# Next add back row 2 into the Canvas file we will upload to avoid errors
# Load original Canvas file again (with full rows)
canvas_original = pd.read_csv("CanvasGrades.csv")

# Extract the original row 1 (index 0)
row_to_insert = canvas_original.iloc[0]

# There are empty columns in this row. In future versions, PANDAS will fill with NA
# on empty values. Will probably need to go back and replace NA with nothing after concatenation.

# Insert it as the new row 1 (index 0), shifting the others down
canvas = pd.concat(
    [canvas.iloc[:0], row_to_insert.to_frame().T, canvas.iloc[0:]],
    ignore_index=True
)

print("✅ Row 1 inserted.")

✅ Row 1 inserted.


/tmp/ipykernel_5995/3210766491.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  canvas = pd.concat(


In [49]:
# List of merged Canvas column names (the targets)
merged_columns = list(merge_map.values())

# Also keep identifying columns (e.g., Student, ID)
id_columns = ['Student', 'ID', 'SIS User ID', 'SIS Login ID', 'Section']  # adjust if needed

# Keep only identifying + merged columns
columns_to_keep = [col for col in canvas.columns if col in merged_columns or col in id_columns]

# Filter canvas DataFrame
canvas = canvas[columns_to_keep]

# Then continue with saving:
canvas.to_csv("merged_canvas_grades.csv", index=False)
# Index=False means no need to re-write the row numbers here
print("✅ Saved only merged and identifying columns to 'merged_canvas_grades.csv'")

✅ Saved only merged and identifying columns to 'merged_canvas_grades.csv'
